In [2]:
import qmcpy as qp
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import cProfile
import pstats

In [3]:
%matplotlib inline

In [4]:
def natural(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="natural" ,is_parallel=True)
    points = l.gen_samples(num_ports) 
    return points
def natural_process(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="natural_process")
    points = l.gen_samples(num_ports) 
    return points
def mps(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="mps" ,is_parallel=True)
    points = l.gen_samples(num_ports) 
    return points
def mps_process(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="mps_process",is_parallel=True)
    points = l.gen_samples(num_ports) 
    return points
def linear(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="linear" ,is_parallel=True)
    points = l.gen_samples(num_ports) 
    return points
def linear_process(n, num_ports, seed=None):
    l = qp.Lattice(dimension=n, seed=seed,order="linear_process")
    points = l.gen_samples(num_ports) 
    return points

In [35]:
trials = 1
d_list = [50, 100]
n = 10
data = {'index':d_list}
frame = pd.DataFrame(data)
frame = frame.set_index('index')
names = set()

run_list = [(natural, "natural"),(natural_process,"natural process"),(linear,"linear"),(linear_process,"linear process")]
for method,method_name in run_list:

    name = method_name.split(" ")

    names.add(name[0])

    times = []
    
    for d in d_list:
        profiler = cProfile.Profile()
        profiler.enable()
        for _ in range(trials):
            _ =method(d, 2 ** n)
        profiler.disable()
        
        stats = pstats.Stats(profiler)
            
        process_time = stats.total_tt

        print(f'Process time {process_time} method name {method_name}')
        times.append(process_time)
    frame[method_name] = times        


names = list(names)        


Process time 0.007599200000000002 method name natural
Process time 0.011052199999999998 method name natural
Process time 1.3595032999999999 method name natural process
Process time 1.2624995999999997 method name natural process
Process time 0.004101699999999999 method name linear
Process time 0.0071664999999999975 method name linear
Process time 0.0035678000000000007 method name linear process
Process time 0.007100699999999998 method name linear process


In [67]:
def speed_up(data_frame, element_list, dimensions):
    speed_factor = []
    A = []
    B = []
    for i in element_list:
        for x in dimensions:
            A.append(data_frame[i][x])
            B.append(data_frame[i + " process"][x])
    for a,b in zip(A, B):
        speed_factor.append(b/a)
    order = []
    for i in element_list:
        for x in dimensions:
            order.append((i,x))
    return list(zip(order,speed_factor))


            

        



In [68]:
speed_up(frame, names, d_list)

[0.8698344588829027, 0.9908183911253751, 178.90084482577106, 114.23061471924142]
[('linear', 50), ('linear', 100), ('natural', 50), ('natural', 100)]
[(('linear', 50), 0.8698344588829027), (('linear', 100), 0.9908183911253751), (('natural', 50), 178.90084482577106), (('natural', 100), 114.23061471924142)]


In [43]:
print(frame)

        natural  natural process    linear  linear process
index                                                     
50     0.007599         1.359503  0.004102        0.003568
100    0.011052         1.262500  0.007166        0.007101
